CNN: 0.98
ABC:

In [1]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
torch.cuda.is_available()
torch.cuda.set_device(0)

In [2]:
mnist_args = dotdict()
mnist_args.train = dotdict()
mnist_args.predict = dotdict()

mnist_args.name = 'mnist'
mnist_args.train.batch_size = 100
mnist_args.predict.batch_size = 1000

In [8]:
args = dotdict()
args.data = mnist_args

args.train_epochs= 3
args.lr = 0.001
args.criterion = 'CE'
args.use_gpu = True

In [9]:
driver = ABC_Driver(args)

Use GPU: cuda:None


In [10]:
driver.train()

epoch: 0, train_loss: 1.8241026322046916
epoch: 1, train_loss: 1.765499724348386
epoch: 2, train_loss: 1.689587230483691


In [11]:
driver.metric()

0.7701

In [7]:
driver.model.ABC_2D.weights

Parameter containing:
tensor([[[-0.8019,  2.1893, -1.3247,  ..., -0.3531,  0.5268,  1.3728],
         [-0.3335, -0.6106,  1.4418,  ..., -1.8536,  0.0585,  0.5964],
         [-0.4316,  0.6728,  1.6959,  ...,  0.3626, -1.6408, -1.0013],
         ...,
         [ 0.3553,  0.3410,  0.3894,  ..., -1.0512,  1.7624,  0.0202],
         [ 0.0676,  1.3749, -0.0179,  ..., -0.4637, -1.1624,  1.8597],
         [-0.6805,  0.2535,  0.2879,  ...,  1.2075, -0.2583,  1.3833]],

        [[ 0.8344,  2.1367, -0.8534,  ..., -0.1845, -0.1901,  1.0107],
         [ 0.2573, -0.4902,  1.3539,  ...,  0.2100,  0.4245, -0.6263],
         [ 1.3222, -1.2416, -0.4546,  ..., -0.4021, -1.1242,  2.3080],
         ...,
         [ 0.4948,  0.2122,  1.1025,  ...,  0.8985,  0.0100, -1.2998],
         [ 0.6499,  0.8059,  0.5366,  ...,  1.7045,  1.6267, -0.0765],
         [ 0.5883, -0.4535,  0.8322,  ...,  1.3949,  0.4625, -0.3759]],

        [[-0.2655,  0.1771,  1.5713,  ...,  0.7416, -0.8200, -0.1557],
         [-1.3435,  1.6

In [11]:
train = torch.utils.data.DataLoader(torchvision.datasets.MNIST('../../data/ABC/mnist', train=True, download=True, 
                                    transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                torchvision.transforms.Normalize((0.1307,), (0.3081,))])),
                                    batch_size=args.data.train.batch_size, shuffle=True)

In [15]:
def get_cov_hashTable(data_mat:torch.tensor):
    data_shape = data_mat.shape
    data_mat=  data_mat.reshape(data_shape[0], -1, data_shape[-2], data_shape[-1])
    B,C,H,W = data_mat.shape
    idx_list_channels = []
    for channel in range(C):
        data_mat1 = data_mat[:,channel,:]
        Num, Hi, Wi = data_mat1.shape
        data_mat1 = data_mat1.reshape(-1, Hi*Wi)
        cov  = torch.cov(data_mat1.T).abs()
        val,idx = torch.topk(cov,k=9,dim=0,sorted=True,largest=True)
        idx_expanded = torch.unsqueeze(idx.T, axis = 1)
        idx_list_channels.append(idx_expanded)
    full_idx_list = torch.concat(idx_list_channels, axis=1)
    return {i: row for i, row in enumerate(full_idx_list)}

In [16]:
hash = get_cov_hashTable(train.dataset.data)

In [17]:
hash

{0: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 1: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 2: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 3: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 4: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 5: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 6: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 7: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 8: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 9: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 10: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 11: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 12: tensor([[ 13,  68, 553, 496, 554, 524, 552,  96, 515]]),
 13: tensor([[ 13, 554, 496, 553, 515, 458, 516, 495, 459]]),
 14: tensor([[ 42, 100, 481, 509,  99, 526, 454, 538,  14]]),
 15: tensor([[ 42, 100, 481, 509,  99, 526, 454, 538,  14]]),
 16: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 17: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 18: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 19: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 20: tensor([[8, 4, 1, 7, 5, 3, 0, 2, 6]]),
 21: tensor([[

In [47]:
def img_reconstruction(hashtable, img, multi_channel=True):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    B,C,H,W = img.shape
    sigle_img_idx = torch.empty((0))
    for key in hashtable.keys():
        idx = hashtable[key]
        for channel in range(idx.shape[0]):
            sigle_img_idx = torch.concat([sigle_img_idx, idx[channel] + H*W*channel])
    all_idx = torch.empty((0))
    for batch in range(B):
        all_idx = torch.concat([all_idx, sigle_img_idx + H*W*C*batch])
    print(all_idx.long())
    final_img = img.take(all_idx.long()).reshape(B, H*W, -1).permute(1,2,0)
    return final_img

In [48]:
img = torch.arange(0,78400).reshape(100,1,28,28)

In [49]:
final = img_reconstruction(hash, img)

tensor([    8,     4,     1,  ..., 77616, 77618, 77622])


In [51]:
final[0]

tensor([[    8,   792,  1576,  2360,  3144,  3928,  4712,  5496,  6280,  7064,
          7848,  8632,  9416, 10200, 10984, 11768, 12552, 13336, 14120, 14904,
         15688, 16472, 17256, 18040, 18824, 19608, 20392, 21176, 21960, 22744,
         23528, 24312, 25096, 25880, 26664, 27448, 28232, 29016, 29800, 30584,
         31368, 32152, 32936, 33720, 34504, 35288, 36072, 36856, 37640, 38424,
         39208, 39992, 40776, 41560, 42344, 43128, 43912, 44696, 45480, 46264,
         47048, 47832, 48616, 49400, 50184, 50968, 51752, 52536, 53320, 54104,
         54888, 55672, 56456, 57240, 58024, 58808, 59592, 60376, 61160, 61944,
         62728, 63512, 64296, 65080, 65864, 66648, 67432, 68216, 69000, 69784,
         70568, 71352, 72136, 72920, 73704, 74488, 75272, 76056, 76840, 77624],
        [    4,   788,  1572,  2356,  3140,  3924,  4708,  5492,  6276,  7060,
          7844,  8628,  9412, 10196, 10980, 11764, 12548, 13332, 14116, 14900,
         15684, 16468, 17252, 18036, 18820, 19604, 

In [22]:
train_data = torchvision.datasets.MNIST('../../data/ABC/mnist', train=False, download=True,
                                                  transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                            torchvision.transforms.Normalize((0.1307,), (0.3081,))]))

In [23]:
train_data.

Dataset MNIST
    Number of datapoints: 10000
    Root location: ../../data/ABC/mnist
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [20]:
a = torch.tensor([[[1,2],[3,4]],[[1,2],[3,4]],[[1,2],[3,4]]])

In [21]:
torch.matmul(a,a)

tensor([[[ 7, 10],
         [15, 22]],

        [[ 7, 10],
         [15, 22]],

        [[ 7, 10],
         [15, 22]]])